<a href="https://colab.research.google.com/github/maryamyousefizadeh/BERT-NER/blob/master/BERT_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get -qq install -y libarchive-dev

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.6_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.6) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.6) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:
!git clone -l -s  git://github.com/kamalkraj/BERT-NER.git cloned-repo-BERT-NER
%cd cloned-repo-BERT-NER
!ls

Cloning into 'cloned-repo-BERT-NER'...
remote: Enumerating objects: 246, done.
remote: Total 246 (delta 0), reused 0 (delta 0), pack-reused 246
Receiving objects: 100% (246/246), 1.67 MiB | 3.31 MiB/s, done.
Resolving deltas: 100% (125/125), done.
/content/cloned-repo-BERT-NER
api.py	 cpp-app  img	       README.md	 run_ner.py
bert.py  data	  LICENSE.txt  requirements.txt


In [4]:
#Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
      -H 'Accept: application/vnd.github.v3.raw' \
      --location https://api.github.com/repos/kamalkraj/BERT-NER/contents/data/test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  730k  100  730k    0     0  1172k      0 --:--:-- --:--:-- --:--:-- 1170k


In [5]:
!curl --remote-name \
      -H 'Accept: application/vnd.github.v3.raw' \
      --location https://api.github.com/repos/kamalkraj/BERT-NER/contents/data/train.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   381  100   381    0     0    915      0 --:--:-- --:--:-- --:--:--   915


In [6]:
!curl --remote-name \
      -H 'Accept: application/vnd.github.v3.raw' \
      --location https://api.github.com/repos/kamalkraj/BERT-NER/contents/data/valid.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  808k  100  808k    0     0  1081k      0 --:--:-- --:--:-- --:--:-- 1080k


In [7]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import variable
from torch.utils.data import  Dataset,Dataset


## Mount Drive into Colab
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
!pip3 install pytorch-transformers


     |████████████████████████████████| 184kB 8.2MB/s 
     |████████████████████████████████| 1.1MB 14.4MB/s 
     |████████████████████████████████| 890kB 28.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c754a2c3ce1217803e501f24902a9355c6c6670c1cdb5f88b6a77f064d055f8d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [9]:
!pip install  torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1MB 20kB/s 
     |████████████████████████████████| 8.8MB 643kB/s 
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
  Found existing installation: torchvision 0.6.0+cu101
    Uninstalling torchvision-0.6.0+cu101:
      Successfully uninstalled torchvision-0.6.0+cu101


In [10]:
!pip install seqeval==0.0.5

In [11]:
!pip install tqdm==4.31.1

     |████████████████████████████████| 51kB 4.1MB/s 
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.31.1 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [12]:
!pip install nltk==3.4.5

     |████████████████████████████████| 1.5MB 8.1MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449906 sha256=db8bf6f86c1cad9ad8f8b4d7f87eea273eb08f43f47031e28d61653a3fa50c2d
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [13]:
!pip install Flask==1.1.1
!pip install Flask-Cors==3.0.8

     |████████████████████████████████| 102kB 5.5MB/s 
  Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2


In [14]:
!python run_ner.py --data_dir=data/ --bert_model=bert-base-cased --task_name=ner --output_dir=out_base --max_seq_length=128 --do_train --num_train_epochs 5 --do_eval --warmup_proportion=0.1



06/16/2020 03:40:34 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
06/16/2020 03:40:34 - INFO - pytorch_transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmpaqf1fsi2
100% 213450/213450 [00:00<00:00, 578988.44B/s]
06/16/2020 03:40:35 - INFO - pytorch_transformers.file_utils -   copying /tmp/tmpaqf1fsi2 to cache at /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
06/16/2020 03:40:35 - INFO - pytorch_transformers.file_utils -   creating metadata file for /root/.cache/torch/pytorch_transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
06/16/2020 03:40:35 - INFO - pytorch_transformers.file_utils -   removing te